Pick train, val, test scenes uniformly based on Completeness.

In [ ]:
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append('/home/ovoinov/work/sk3d/dev.sk_robot_rgbd_data/src')
from skrgbd.evaluation.pathfinder import Pathfinder as EvalPathfinder
from skrgbd.evaluation.stats_db import StatsDB
from skrgbd.utils.logging import tqdm

In [ ]:
eval_pathfinder = EvalPathfinder('/home/ovoinov/remotes/root.zhores/gpfs/data/gpfs0/3ddl/projects/sk3d')
stats_db = eval_pathfinder.evaluation.all_stats
stats_db = StatsDB(stats_db, mode='r')

In [ ]:
experiments = [
    dict(method='acmp', version='v1.0.0', camera='tis_right', light='ambient@best'),
    dict(method='colmap', version='v1.0.0', camera='tis_right', light='ambient@best'),
    dict(method='vismvsnet', version='v1.0.0_trained_on_blendedmvg', camera='tis_right', light='ambient@best'),
    dict(method='unimvsnet', version='v1.0.0_authors_checkpoint_v1', camera='tis_right', light='ambient@best'),
#     dict(method='tsdf_fusion', version='v0.0.0_v3', camera='kinect_v2', light='none'),
#     dict(method='surfel_meshing', version='v1.0.0_v3', camera='kinect_v2', light='none'),
#     dict(method='routed_fusion', version='v0.0.0_v3', camera='kinect_v2', light='none'),
    dict(method='neus', version='v1.0.0_v1', camera='tis_right', light='ambient@best'),
#     dict(method='azinovic22neural', version='v0.0.0_v3', camera='kinect_v2@tis_right', light='ambient@best'),
]
query = ' or '.join(' and '.join(f'{k} == "{v}"' for (k, v) in experiment.items()) for experiment in experiments)

colors = {
    'colmap':'tab:blue',
    'acmp':'tab:orange',
    'vismvsnet':'tab:green',
    'unimvsnet':'tab:red',
#     'tsdf_fusion':'tab:purple',
#     'surfel_meshing':'tab:brown',
#     'routed_fusion':'tab:pink',
    'neus': 'tab:olive',
#     'azinovic22neural': 'tab:cyan',
}

method_labels = {'tsdf_fusion': 'TSDF Fusion',
                 'surfel_meshing': 'SurfelMeshing',
                 'routed_fusion': 'RoutedFusion',
                 'colmap': 'COLMAP',
                 'acmp': 'ACMP',
                 'vismvsnet': 'VisMVSNet',
                 'unimvsnet': 'UniMVSNet',
                 'neus': 'NeuS',
                 'azinovic22neural': 'Neural RGBD'}

measure_labels = {'completeness': 'Recall', 'accuracy': 'Precision'}

methods = [
    'colmap',
    'acmp',
    'vismvsnet',
    'unimvsnet',
#     'tsdf_fusion',
#     'routed_fusion',
#     'surfel_meshing',
    'neus',
#     'azinovic22neural',
]

In [ ]:
threshold = 5e-4
thresholds = pd.read_sql('select threshold from completeness where method="colmap" and scene="dragon"', stats_db.conn).threshold.to_numpy()
threshold = thresholds[np.abs(thresholds - threshold).argmin()]; del thresholds

In [ ]:
scale = 2
fig_h = scale * 8.90828111117  # CVPR \textheight in inches
fig_h *=.98
plt.rc('font', size=80, family='Times New Roman')
fontsize = 5 * scale
w_to_h = .7
for measure in [
    'completeness',
#     'accuracy'
]:
    db = pd.read_sql(
        f'select method,version,scene,camera,light,value from {measure} where threshold="{threshold}"', stats_db.conn,
    ).query(query).drop(columns=['version', 'camera', 'light'])
    db = db.set_index(['scene', 'method']).sort_index()

    sorting = db.groupby(level='scene').max().sort_values('value').index
    db = db.reindex(sorting, level=0)

    height = .8
    ys = np.arange(len(sorting))

    fig, ax = plt.subplots(figsize=(fig_h * w_to_h, fig_h), dpi=200 / scale)
    for method in methods:
        ax.barh(ys[0], [0], color=colors[method], label=method_labels[method])

    for scene_i, scene in enumerate(sorting):
        scene_data = db.loc[scene].sort_values('value', ascending=False)
        for data in scene_data.iloc:
            method = data.name
            ax.barh(
                ys[scene_i: scene_i+1],
                [data.value * 100],
                color=colors[method],
                capsize=1,
                height=height,
            )

    ax.set_yticks(ys)
    ax.set_yticklabels(sorting)
    ax.set_ylim(-1, 107)

    ax.set_xticks(range(0, 101, 5))
    ax.set_xlim(25, 100)
    ax.grid(alpha=1, axis='x', color='#EEE')
    ax.tick_params(axis='both', which='major', labelsize=fontsize, top=True, labeltop=True)
    ax.tick_params(axis='both', which='minor', labelsize=fontsize)

    plt.subplots_adjust(0, 0, 1, 1, wspace=0, hspace=0)
    legend = ax.legend(loc='lower right', bbox_to_anchor=(1, 0), fontsize=fontsize)
    plt.close()

In [ ]:
scenes_n = 107
test_scenes_n = 20
val_scenes_n = round((scenes_n - test_scenes_n) * .2)

In [ ]:
# The scenes I manually picked for the second submission
test_scenes = {
    'small_wooden_chessboard',
#     'dragon',
    
    'white_box',
    'dumbbells',
    'green_flower_pot',
    'enamel_bowl',
    'rose_bowl_with_cover',
    'white_iron',
    'white_mixer',
    'green_funnels',
    'pink_wheeled_alarm_clock',
    'white_fox_figurine',
    'ceramic_girl_in_hat',
    'white_ceramic_elephant',
}

# Sample additional scenes uniformly between the simplest scene and the scene simplest among the hard ones 
ids = np.linspace(
    sorting.get_loc('ceramic_girl_in_hat'),
    sorting.get_loc('small_wooden_chessboard'),
    test_scenes_n - len(test_scenes) + 2
).round().astype(int)
test_scenes.update([sorting[i] for i in ids])

# Replace some scenes manually for more uniform distribution of surface properties
test_scenes = {
    'brown_relief_pot', # 'black_dewar',
    'ceramic_girl_in_hat',
    'dumbbells',
    'enamel_bowl',
    'green_flower_pot',
    'green_funnels',
    'painted_samovar',
    'red_moai', # 'pink_boot',
    'pink_wheeled_alarm_clock',
    'white_castle_towers', #'red_telescope',
    'rose_bowl_with_cover',
    'jewelry_box', # 'rubiks_cube',
    'small_wooden_chessboard',
    'white_bicycle_dewar',
    'white_box',
    'white_ceramic_elephant',
    'white_fox_figurine',
    'white_iron',
    'white_mixer',
    'dragon', #'white_toaster',
}
len(test_scenes)

In [ ]:
for t in ax.get_yticklabels():
    if t._text in test_scenes:
        t.set_color('red')
    else:
        t.set_color('black')
fig

In [ ]:
train_val_index = sorting.drop(test_scenes)

ids = np.linspace(0, len(train_val_index) - 1, val_scenes_n).round().astype(int)
val_scenes = set([train_val_index[i] for i in ids])

# Replace some scenes manually
val_scenes = {
    'painted_cup', # 'black_dewar',
    'black_lunch_box',
    'dutch_oven',
    'green_lumpy_ball',
    'grey_braided_box',
    'large_candles',
    'large_coral_backpack',
    'orange_cash_register',
    'orange_mini_vacuum',
    'plastic_micro_cover',
    'plush_bear',
    'red_ceramic_fish',
    'red_table_lamp',
    'vintage_camera',
    'white_ceramic_monkey',
    'white_pot',
    'wooden_trex',
}